## 经典数据表示

In [ ]:
浮点数（IEEE 754 normal/subnormal）


浮点规范 (bit for sign)(bits for exponent)(bits for mantissa)的方式，来近似实数. 采用二进制，它在数值上相等的实数值 = sign * [1 + fraction] * 2^(magnitude)  
  
fraction 是用 bits for mantissa 计算出来的。bits for mantissa，比如 FP64 规范 有 52 位bits, 二进制下这些 bits 的第1位到第52位分别代表 1/2 ,..., 1/2^52。累加后，加 1 (默认隐含)  
  
magnitude 是用 bits for exponent 计算出来的。bits for exponent，比如 FP64 规范 有 11 位bits, 二进制下这些 bits 可算出一个十进制指数. 减去偏置 bias（根据具体类型确定）  


Q1：fraction 为什么要加 1?  
A1：加 1 ，使得最终数值等于 1.xxx * 2^E 的形式，如果没有加 1，那么最终数值等于 0.xxx * 2^E 的形式。说明 mantissa 省略了科学计数法底数里的 1，节省出来的bits扩大了表示范围  
  
Q2：magnitude 为什么要减 bias?  
A2：简而言之是为了表示出 负指数，用以表示绝对值很小很小的数（规范的尺寸下限）。
比如 FP64 exponent部分有 11 位，按理最大(11个1)二进制表示2^11-1=2047。为了表示负指数，且正负平等，那么2047的一半1023作为bias，通过减去 bias 得到 负指数。
所以最大的正指数是 2046-1023 = 1023（全1的2047在 FP64 规范 里代表正无穷了），最小的负指数是 1-1023 = 1022（全0的exponent不符合normal 规约, 会触发subnormal 规约），
配合 1.fractional 的尾数，range上限是 1.11--1 * 2^1023, range下限是 0.00--1 * 2^-1022，由此确定了这个 FP64规范 的 尺寸(magnitude)

In [ ]:
规范 都分成3部分，符号位，指数和尾数。不同 规范 只不过是指数位和尾数位的长度不一样。

解析一个浮点数就5条规则

如果指数位全零，尾数位是全零，那就表示0
如果指数位全零，尾数位是非零，就表示一个很小的数（subnormal），计算方式 (−1)^signbit × 2^−126 × 0.fractionbits
如果指数位全是1，尾数位是全零，表示正负无穷
如果指数位全是1，尾数位是非零，表示不是一个数NAN
剩下的计算方式为 (−1)^signbit × 2^(exponentbits − bias) × 1.fractionbits

In [ ]:
FP64 double 8bytes 64bits
1位符号+11位指数+52位小数
range: 10^-308 至 10^308，precision：15-17位小数精度
IEEE 754定义，大多数C/C++系统里的double类型，x86CPU支持，GPU内支持但一般严重限制性能（除非一些专业卡）。一些科学计算（特别是迭代的）必须使用这个 FP64 不然累积误差过大

In [ ]:
FP32 float 4bytes 32bits
1位符号+8位指数+23位小数
range: 10^-38 至 10^38，precision：6-9位小数精度
IEEE 754定义，大多数C/C++系统里的float类型，x86CPU支持，GPU支持，是长期以来深度学习/神经网络的标准类型

理想情况下，训练和推理中涉及的数字都应该用 FP32 规范

In [ ]:
FP16 2bytes 16bits
1位符号+5位指数+10位小数
range: 10^-5 至 65504，precision：4位小数精度
英伟达于2022年定义，C/C++ 不支持，x86 CPU 不支持，GPU良好支持。容易上溢（超出range上限）/下溢（低于range下限）

理想情况下，训练和推理中涉及的数字都应该用 FP32 规范。但考虑到性能（FP16占据更少显存、速度更快），且精度没那么重要，实践中通常使用混合精度：
训练：
    使用 FP32 精度的weights作为 精确的 master weights。
    使用 FP16/BF6 等规范的权重作前向计算、后向传播（即梯度是 FP16/BF16）
    使用 FP16/BF16 的梯度去更新 FP32 的主权重
推理：
    FP16 规范的weights就有不错的效果


FP16 是一种独立的数据类型，从 FP32 转换到 FP16，需要显式类型转换，才能利用 FP16 的优势（内存带宽占用小，故更快的数据传输和执行）。

## 梯度上溢/下溢，以及解决办法

In [ ]:
由于 FP16 容易上溢(超出65504)/下溢(比10^-5更接近0)，这样若梯度发生了上下溢，那么上溢会导致梯度爆炸NaN，要么下溢导致权重无法更新(0梯度)。  

原生梯度下溢问题：
梯度缩放（loss scaling trick） 可以缓解。  
计算 loss 时，使用系数 a（一般是2的高次），乘以 original loss，得到 a_loss。对 a_loss 作反向传播后，得到的梯度 a_grad 也被同步缩放，所以要对 a_grad 除以 a。   

梯度缩放（loss scaling trick）过程中如果出现了 梯度上溢，即乘完 a 之后，梯度超出上限，这个时候动态梯度缩放机制会起作用：优化器会跳过当前的参数更新，然后减小系数 a。  
在下一次迭代中，使用缩小后的 a 作梯度缩放。如果没问题了，逐渐增大 a。   
  
  

  
原生梯度上溢问题：
原生梯度上溢/爆炸问题 有多种缓解办法：  
1. 混合精度训练，会涉及到上溢的操作，使用更大 magnitude 的 FP32 规范来算。
2. 梯度裁剪
3. 归一化层, batch norm 或 layer norm，归一化层可以有效稳定激活函数的范围，保持数值稳定
4. 更稳定的激活函数(relu > tanh/sigmoid)，更好的初始化策略、学习率调度等
5. 采用 派生的数据规范 比如 BF16 / TF32 等

## 派生数据表示

In [ ]:
截断 FP32，保留指数部分以保持 magnitude，但由于尾数部分被截断，所以损失 precision。

有两种截断方法：
TF32（从32位截断到19位，保留了FP32的magnitude，有 FP16 的precision）
BF16（从32位截断到16位，保留了FP32的magnitude，有 不如FP16的precision）

In [ ]:
TF32 19bits
1位符号+8位指数+10位小数，是 FP32 直接截断尾数部分（降低precision）至对齐FP16，保留指数部分（保留magnitude），总位数是19
range: 10^-38 至 10^38，precision：4位小数精度
英伟达 定义，C/C++ 不支持，x86 CPU 不支持，N卡A系列/cuda11 开始支持。

与 FP32 一样的范围，故上下溢问题比 FP16 要轻很多。精度和 FP16一样

TF32有一个优点：无需显式类型转换
它在cuda编译器/tensor core内部“深度黑箱式优化”，使得外部看来，它就是“低精度、高matmul性能的FP32”，输入输出都是 正经的FP32，只有内部部分操作时被转换了。
具体来说，
    1. 在外部接口看来，TF32和FP32都是torch.float32。
    2. 在模型执行底层，weights本身、以及非矩阵乘法matmul操作（比如激活函数等）都保持FP32。这些地方需要精度。
    3. 在进入tensor core执行的操作时（比如matmul），这些 FP32 会被转化成 TF32 进行计算，算完之后再转换成 FP32

In [ ]:
BF16 2bytes 16bits
1位符号+8位指数+7位小数，是 FP32 直接截断尾数部分（降低precision），保留指数部分（保留magnitude），使得总位数对齐 FP16
range: 10^-38 至 10^38，precision：3位小数精度
谷歌 定义，C/C++ 不支持，x86 CPU 不支持，N卡A系列/cuda11 开始支持。

与 FP32 一样的范围，故上下溢问题比 FP16 要轻很多。但是 BF16 的精度比 FP16 还低

过低的精度会导致收敛问题，包括但不限于误差在多次迭代中累积、小精度更新不稳定等。
解决办法包括 降低学习率以稳定收敛、增大batch_size以减少噪声、调整某些对精度敏感的layer、用稳定收敛的优化器、梯度裁剪等等。

BF16 是一种独立的数据类型，从 FP32 转换到 BF16，需要显式类型转换，才能利用 BF16 的优势（内存带宽占用小，故更快的数据传输和执行）。

In [ ]:
Int8